In [27]:
# import requests
from urllib.request import urlopen
import time
from bs4 import BeautifulSoup
import pandas as pd

In [28]:
url = 'https://iniciativas.chileconvencion.cl/m/iniciativa_popular/'
html = urlopen(url)

if html.status != 200:
    print("Error fetching page")
    exit()
else:
    content = html
print(content)

In [29]:
soup = BeautifulSoup(html,"html.parser")
type(soup)
print(soup.prettify())
len(list(soup.descendants))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



83307

In [30]:
numero_propuesta_raw = soup.select('div.card > div > h2')

numero_propuesta = [nombre.text for nombre in numero_propuesta_raw]
print(len(numero_propuesta))
# numero_propuesta[0]
print(numero_propuesta)
print(type(numero_propuesta))

2495
['Propuesta nº 71002', 'Propuesta nº 75690', 'Propuesta nº 52642', 'Propuesta nº 11542', 'Propuesta nº 73970', 'Propuesta nº 56170', 'Propuesta nº 64102', 'Propuesta nº 59582', 'Propuesta nº 75830', 'Propuesta nº 73766', 'Propuesta nº 586', 'Propuesta nº 53070', 'Propuesta nº 53782', 'Propuesta nº 53130', 'Propuesta nº 42034', 'Propuesta nº 52946', 'Propuesta nº 15174', 'Propuesta nº 49942', 'Propuesta nº 73994', 'Propuesta nº 66450', 'Propuesta nº 71522', 'Propuesta nº 65082', 'Propuesta nº 33314', 'Propuesta nº 60670', 'Propuesta nº 28646', 'Propuesta nº 35854', 'Propuesta nº 49206', 'Propuesta nº 36190', 'Propuesta nº 56694', 'Propuesta nº 47066', 'Propuesta nº 12102', 'Propuesta nº 62494', 'Propuesta nº 52118', 'Propuesta nº 49734', 'Propuesta nº 47918', 'Propuesta nº 1530', 'Propuesta nº 70434', 'Propuesta nº 41142', 'Propuesta nº 15138', 'Propuesta nº 48842', 'Propuesta nº 37338', 'Propuesta nº 14882', 'Propuesta nº 67930', 'Propuesta nº 69378', 'Propuesta nº 65978', 'Propue

In [31]:
nombre_autor_raw = soup.select('div.card > div.autor')


nombre_autor = [nombre.text for nombre in nombre_autor_raw]
print(len(nombre_autor))
# nnombre_autor[0]
# print(nombre_autor)

2495


In [32]:
iniciativa_raw = soup.select('div.card > div > h1 > a')
nombre_iniciativa = [nombre.text.lower().capitalize() for nombre in iniciativa_raw]
#nombre_iniciativa_small = nombre_iniciativa.small()

print(len(nombre_iniciativa))
# seccion[0]
# print(nombre_iniciativa)

2495


In [33]:
seccion_raw = soup.select('div.card > div > h1 > a')


seccion = [row.text for row in seccion_raw]
print(len(seccion))
# seccion[0]
# print(nombre_iniciativa)

2495


In [34]:
import re


apoyos_html = soup.select('div.card > div:nth-of-type(3)')

conteo_apoyos_raw = [str(valor.text) for valor in apoyos_html]
# print(conteo_apoyos_raw)
# print(len(conteo_apoyos_raw))

# for idx, val in enumerate(conteo_apoyos_raw):
#     print(idx, val)

conteo_apoyos_unflatened = []
conteo_apoyos_unflatened += (re.findall(r'([0-9]+\.?[0-9]*|\.[0-9]+)', item_apoyo) for item_apoyo in conteo_apoyos_raw)

conteo_apoyos_flattened = [val for sublist in conteo_apoyos_unflatened for val in sublist]

# for item_apoyo in list_apoyos:
#   if item_apoyo == re.findall(r'\d\.?\d?\K?', item_apoyo)
#     print(item_apoyo)

# for item_apoyo in list_apoyos:
#   conteo_apoyos_unflatened += re.findall(r'\d+\w+', item_apoyo)



# for idx, val in enumerate(conteo_apoyos_unflatened):
#     print(idx, val)

# print(conteo_apoyos_flattened)
# print(len(conteo_apoyos_flattened))
# print(type(conteo_apoyos_flattened))


In [35]:
multiplier = []

# print(conteo_apoyos_raw)

for conteo_raw in conteo_apoyos_raw:
  if re.findall(r'\d+K', conteo_raw):
    multiplier += "T"
  else:
    multiplier += "F"

# print(multiplier)
len(multiplier)
print(type(multiplier))

<class 'list'>


In [36]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
          {
              'Nombre': nombre_iniciativa,
              'Apoyos': conteo_apoyos_flattened,
              'Multiplicador': multiplier,
              'Autor': nombre_autor,
              'Seccion' : seccion
          }

    )


df = df.astype({"Apoyos": float})



# for data in df[1]:
#   print(data)
# pd.to_numeric(df[conteo_apoyos_flattened])

# if df["Multiplicador"] == "T":
#   df["Conteo Apoyos"].apply(x * 1000)


# print(df)
# df.shape
# len(df)
# print(df.head(4))
# df.info()
# df.columns
# print(df.loc[:,'Multiplicador'])
print(df['Apoyos'])
df['Apoyos'] = np.where((df.Multiplicador == 'T'), (1000 * df.Apoyos) ,df.Apoyos)
type(df['Apoyos'])
df_sorted = df.sort_values('Apoyos',  ascending=False)
df_slice = df_sorted[:50]
df_slice.head()

0        67.0
1        36.0
2         9.0
3        51.0
4         6.0
        ...  
2490      9.2
2491     18.0
2492    136.0
2493    401.0
2494     69.0
Name: Apoyos, Length: 2495, dtype: float64


,Nombre,Apoyos,Multiplicador,Autor,Seccion
2361,Con mi plata no - defiende tus ahorros previsi...,44000.0,T,Por Con Mi Plata NO,CON MI PLATA NO - DEFIENDE TUS AHORROS PREVISI...
1332,Cannabis a la constitución ahora: por el derec...,38000.0,T,Por Activismo Cannábico Chile,Cannabis a la Constitución Ahora: Por el derec...
2458,Libre derecho sobre la propiedad privada,35000.0,T,Por María Chacón W.,Libre derecho sobre la propiedad privada
2252,Será ley,29000.0,T,Por Asamblea Permanente por la Legalización de...,Será Ley
1788,Banco central autónomo,25000.0,T,Por Banco Central Autónomo,Banco Central Autónomo


In [37]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.rcParams.update({'font.size': 26})
# fig = plt.figure(figsize=(10, 22), dpi=80)
# ax = fig.add_axes([0,0,1,1])

# # Invert axis
# plt.gca().invert_yaxis()

# ax.barh(df_slice['Nombre'],df_slice['Apoyos'])
# plt.style.use('ggplot')
# plt.title('Top 20 Iniciativas Populares', loc ='left', fontsize=32)
# plt.ylabel('Inicitiva')
# plt.xlabel('Numero de Apoyos (miles)')

# # Add Text watermark
# fig.text(0.5, 0.1, 'by e-trolls', fontsize = 40, color ='green', ha ='center', va ='center', alpha = 0.6, rotation='0')

# plt.show(fig)

# plt.savefig('foo.png')

In [38]:
# import base64
# from io import BytesIO

# tmpfile = BytesIO()
# fig.savefig(tmpfile, format='png')
# encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')
# # encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8') <img src='data:image/png;base64,{{}}'>

# html = 'Some html head' + '<img src=\'data:image/png;base64,{}\'>'.format(encoded) + 'Some more html'

# with open('test.html','w') as f:
#    f.write(html)

In [73]:
import plotly.express as px

fig = px.bar(df_slice, 
             x="Apoyos",
             y="Nombre",
             text="Apoyos",
             title="APOYOS A INICIATIVAS POPULARES",
             hover_data=["Autor",
                         "Seccion"],
             orientation='h',
             width=1200,
             height=1000)

fig.update_yaxes(autorange='reversed')

fig.update_traces(texttemplate='%{text:.2s}', 
                  textposition='outside')

fig.update_layout(
    font_family="Times New Roman",
    font_size=12,
    font_color="RebeccaPurple",
    xaxis_title="APOYOS (Miles)",
    yaxis_title="INICIATIVAS",
    title_font_family="Times New Roman",
    title_font_color="grey"
)




fig.add_vrect(x0=15000, 
              x1=15000,
              line_width=1,
              line_dash="dash",
              line_color="red")

# fig.update_xaxes(title_font_family="Arial")

fig.show()

fig.write_html("/home/navarro/Documents/iniciativas_populares/grafico_iniciativa_popular.html")